In [1]:
import tweepy
from os import getenv
from dotenv import load_dotenv
from neo4j import GraphDatabase
import pandas as pd
import time

load_dotenv()

True

In [2]:
consumer_key = getenv('consumer_key')
consumer_secret = getenv('consumer_secret')
access_token = getenv('access_token')
access_token_secret = getenv('access_token_secret')

In [6]:
df = pd.read_csv("output_follower.csv", index_col=[0])
df

,id,Name,Screen_Name,description,prifile_image,follower_id
0,1405886849441140739,藤田旭洋（Akihiro Fujita）,afujitabz,CEO of Bluezone inc. / 東大航空宇宙D2 / 未踏AD (2021) ...,https://pbs.twimg.com/profile_images/151408653...,"[1568798216069746689, 780896517859057665, 1052..."
1,1314113770285793280,いわもっち🌺マーケ部長,mocchi_iwa,マーケ部長やってます/ステキって言います/海が好き/沖縄好き/ダイバー兼サーファー兼webマ...,https://pbs.twimg.com/profile_images/156243996...,"[1567132777237123076, 1470039473282961411, 156..."
2,991303435617120257,sin・cos・カモノハシ,takadanobabauec,元天文 元DHK 元ⅡP4 元11クラ,https://pbs.twimg.com/profile_images/143929848...,[]
3,1543154097691705345,Uday kiran,Uday_Sugali,NaN,https://pbs.twimg.com/profile_images/154315416...,"[2504437416, 3155984131, 1331525617]"
4,1190436526175350784,ヨノ フリーランスエンジニア,yono_free,エンジニア歴9年。2児の父\nRails/React/Vue/TypeScript/gola...,https://pbs.twimg.com/profile_images/119043679...,"[38144769, 1470629774930616322, 15641015025702..."
...,...,...,...,...,...,...
1184,735982190,創英あるあるbot,soeiaru2,創英あるあるを思いついたらDMを気軽にください。 載せさせていただきます。 “あるある”と思...,https://pbs.twimg.com/profile_images/246550589...,"[1537394949356142592, 1499108026615476224, 142..."
1185,387729371,わっち,gracemeria,人生に、野遊びを,https://pbs.twimg.com/profile_images/326233015...,[]
1186,553560130,Heima Ueda,van_der_heima,青学 / LAMDAC2016 / 学生団体 参考書宅救便( @sankousho_SKU ...,https://pbs.twimg.com/profile_images/138833666...,[]
1187,311452958,角なし,EllenGrigia,ロンゾの面汚し,https://pbs.twimg.com/profile_images/599558314...,"[2374395042, 2939330144, 2582123257, 174270015..."


In [3]:
NEO4J_URI = getenv('NEO4J_URI')
NEO4J_USERNAME = getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = getenv('NEO4J_PASSWORD')
AURA_INSTANCENAME = getenv('AURA_INSTANCENAME')

In [ ]:
# 日本語を含める項目も加える
# strを加えて文字列に変換すると動く
main_name = "Y10iBobb"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
start = time.time()
# ここから先を検討中
def add_friend(tx, name, screen_name, twitter_id, Name, description, profile_image):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (friend:Person {name:$screen_name, twitter_id: $twitter_id, Name:$Name, description: $description, profile_image: $profile_image})-[:follow]->(a)",
           name=name, screen_name=screen_name, twitter_id=twitter_id, Name=Name, description=description, profile_image=profile_image)

with driver.session() as session:        
    for index, row in df.iterrows():
        twitter_id = row[0]
        Name = str(row[1])
        screen_name = row[2]
        description = str(row[3])
        profile_image= row[4]

        try:
            session.write_transaction(add_friend, main_name, screen_name, twitter_id, Name, description, profile_image)
        except:
            print("error_" + screen_name)

end = time.time()
driver.close()
print(end - start)

In [ ]:
# followerのfollowerを作成するところ

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
start = time.time()

def add_friend_by_twitter_id(tx, name, twitter_id):
    tx.run("MERGE (a:Person {name: $name})"
           "MERGE (friend:Person {twitter_id: $twitter_id})-[:follow]->(a)",
           name=name, twitter_id=twitter_id)

with driver.session() as session:  
    for index, row in df.iterrows():
        main_name = row[2]
        print(main_name)
        followers_str = row[5].strip("[]")
        followers = followers_str.split(",")
        for follower in followers:
            try:
                follower_id = int(follower)
                session.write_transaction(add_friend_by_twitter_id, main_name, follower_id)
            except:
                print("error_", end=" ")
                print(follower_id)
            print(follower)

end = time.time()
driver.close()
print(end - start)